## Токенизация

Задача - разделить предложение на слова или отдельные элементы (знаки препинания, гиперссылки и т.д.), по возможности сохраняя какие-то атрибуты текста.

# Регурярные выражения

В модуле `re` есть недокументированный класс `Scanner`, с помощью которого можно реализовать лексический анализатор. `Scanner` будет искать вхождения паттернов в тексте и на каждое совпадение вызывать соответствующую функцию. В общем случае подобный код неэффективен, лексические анализаторы лучше реализовывать с помощью специальных инструментов - генераторов лексических анализаторов, которые обеспечат анализ за линейное время.

In [1]:
import re

scanner = re.Scanner(
   [(r'(\w+)@(\w+)\.(\w{2,3})', lambda s, x: (x, 'email')),
    (r'[a-zA-Z]+', lambda s, x: (x, 'word')), 
    (r'\d+', lambda s, x: (x, 'digit')),    
    (r'\s+', lambda s, x: (x, 'whitespace')),
    (r'[.,;"!?:]', lambda s, x: (x, 'preposition')),
    ])

##scanner.scan('hello, world 1234 test@example.com')
scanner.scan("Hello world!")

([('Hello', 'word'),
  (' ', 'whitespace'),
  ('world', 'word'),
  ('!', 'preposition')],
 '')

## NLTK
Natural Language Toolkit, библиотека для обработки естественных языков. Она создавалась для учебных целей, но тем не менее приобрела определенную популярность. Реализовано некоторое количество методов токенизации, которые можно использовать для повседневных задач и экспериментов.

In [2]:
import nltk
from nltk.tokenize import wordpunct_tokenize, word_tokenize, TweetTokenizer

nltk.download('punkt')

tweet_tokenize = TweetTokenizer()

sentences = ["Hello world 4.2.", "LA New-York", "Hello world 4.2!", "Say me #hello"]

for sent in sentences:
    print("Sentence: {}".format(sent))
    print("word_tokenize: ", word_tokenize(sent))
    print("wordpunct_tokenize: ", wordpunct_tokenize(sent)),
    print("tweet: ", tweet_tokenize.tokenize(sent))
    print()

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sentence: Hello world 4.2.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/alex/nltk_data'
    - '/home/alex/evo840/work/envs/venv_nlp/nltk_data'
    - '/home/alex/evo840/work/envs/venv_nlp/share/nltk_data'
    - '/home/alex/evo840/work/envs/venv_nlp/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
from nltk.tokenize import sent_tokenize

print(
    sent_tokenize('I like trains. I like cakes. Dr. House, how are you? I like I.B.M.!'))
print(
    sent_tokenize('The world\'s oldest football competition is the FA Cup, which was founded by C. W. Alcock and has been contested by English teams since 1872.'))

['I like trains.', 'I like cakes.', 'Dr. House, how are you?', 'I like I.B.M.', '!']
["The world's oldest football competition is the FA Cup, which was founded by C. W. Alcock and has been contested by English teams since 1872."]


## Ply

Приведем лексического анализатора на `ply`. В данном случае анализатор описывается в классе, могут быть три вида токенов - слова, цифры и пробелы. Для каждого токена в тексте выозвращается необходимая информация - типа, длина смещение:

In [ ]:
from ply.lex import lex, TOKEN

class Lexer:
    tokens = ( 'NUMBER', 'ID', 'WHITESPACE' )
    
    @TOKEN(r'\d{1,5}')
    def t_NUMBER(self, t):
        t.value = int(t.value)
        return t

    @TOKEN(r'\w+')
    def t_ID(self, t):
        return t

    @TOKEN(r'\s+')
    def t_WHITESPACE(self, t):
        pass

    def t_error(self, t):
        pass
    

__file__ = "02.Tokenizers.ipynb"    # make `ply` happy

lexer = lex(object=Lexer())
lexer.input('123 abs 965')
for token in lexer:
    print(token)

LexToken(NUMBER,123,1,0)
LexToken(ID,'abs',1,4)
LexToken(NUMBER,965,1,8)
